In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import metpy.calc 
from metpy.units import units
import math

C:\Users\josia\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# note that A105 is site 106A, didn't want to rename each varaible

In [3]:
A105 = pd.read_csv('/Users/josia/Desktop/Python/StickNet/0106A_IOP3_level2.txt')
A105 = A105[103386:108666]
A105

,Time,T,RH,P,WS,WD
103386,2023-03-25 00:30:17,24.9,76.0,998.8,5.4,163.0
103387,2023-03-25 00:30:18,24.9,76.0,998.8,5.4,165.7
103388,2023-03-25 00:30:19,24.9,76.0,998.8,5.3,166.8
103389,2023-03-25 00:30:20,24.9,76.0,998.8,5.3,175.0
103390,2023-03-25 00:30:21,24.9,76.1,998.8,5.0,164.8
...,...,...,...,...,...,...
108661,2023-03-25 01:58:12,23.4,89.8,1000.4,8.0,227.7
108662,2023-03-25 01:58:13,23.4,89.9,1000.4,7.4,219.0
108663,2023-03-25 01:58:14,23.4,89.8,1000.5,7.2,227.1
108664,2023-03-25 01:58:15,23.4,89.9,1000.5,8.6,219.9


In [4]:
## Smoothing ##

In [5]:
T = np.array(A105['T'])
RH = np.array(A105['RH'])
P = np.array(A105['P'])
WS = np.array(A105['WS'])
WD = np.array(A105['WD'])

temp = np.zeros(len(T))
rh = np.zeros(len(RH))
p = np.zeros(len(P))
wind_speed = np.zeros(len(WS))
wind_direction = np.zeros(len(WD))

temp[:] = np.nan
rh[:] = np.nan
p[:] = np.nan
wind_speed[:] = np.nan
wind_direction[:] = np.nan

for i in range(2, len(A105) - 2, 1):
    temp[i] = np.nanmean(np.array([T[i-2], T[i-1], T[i], T[i+1], T[i+2]]))
    rh[i] = np.nanmean(np.array([RH[i-2], RH[i-1], RH[i], RH[i+1], RH[i+2]]))
    p[i] = np.nanmean(np.array([P[i-2], P[i-1], P[i], P[i+1], P[i+2]]))
    wind_speed[i] = np.nanmean(np.array([WS[i-2], WS[i-1], WS[i], WS[i+1], WS[i+2]]))
    wind_direction[i] = np.nanmean(np.array([WD[i-2], WD[i-1], WD[i], WD[i+1], WD[i+2]]))

smooth_5s_A105 = pd.DataFrame(data = [np.array(A105['Time']), temp, rh, p, wind_speed, wind_direction], 
                             index = ['Time', 'Temp', 'RH', 'Pressure', 'Wind_Speed', 'Wind_Direction']).transpose()
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN
2,2023-03-25 00:30:19,24.9,76.02,998.8,5.28,167.06
3,2023-03-25 00:30:20,24.9,76.04,998.82,5.14,169.62
4,2023-03-25 00:30:21,24.9,76.06,998.84,5.08,175.9
...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.4,89.82,1000.46,7.66,221.62
5276,2023-03-25 01:58:13,23.4,89.84,1000.46,7.88,221.5
5277,2023-03-25 01:58:14,23.4,89.86,1000.46,7.78,220.66
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN


In [6]:
lon_A105 = -90.9653
lat_A105 = 32.9786
smooth_5s_A105['105A Lon'] = np.full(5280, lon_A105)
smooth_5s_A105['105A Lat'] = np.full(5280, lat_A105)
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786
2,2023-03-25 00:30:19,24.9,76.02,998.8,5.28,167.06,-90.9653,32.9786
3,2023-03-25 00:30:20,24.9,76.04,998.82,5.14,169.62,-90.9653,32.9786
4,2023-03-25 00:30:21,24.9,76.06,998.84,5.08,175.9,-90.9653,32.9786
...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.4,89.82,1000.46,7.66,221.62,-90.9653,32.9786
5276,2023-03-25 01:58:13,23.4,89.84,1000.46,7.88,221.5,-90.9653,32.9786
5277,2023-03-25 01:58:14,23.4,89.86,1000.46,7.78,220.66,-90.9653,32.9786
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786


In [7]:
path = '/Users/josia/Desktop/Python/NSSL Internship/'
meso = pd.read_csv(path + 'Data/mesodata.csv')[0:5280]
smooth_5s_A105['Meso Lon'] = meso['Meso Lon']
smooth_5s_A105['Meso Lat'] = meso['Meso Lat']
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat,Meso Lon,Meso Lat
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-91.374930,32.679903
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-91.374551,32.679969
2,2023-03-25 00:30:19,24.9,76.02,998.8,5.28,167.06,-90.9653,32.9786,-91.374171,32.680034
3,2023-03-25 00:30:20,24.9,76.04,998.82,5.14,169.62,-90.9653,32.9786,-91.373792,32.680099
4,2023-03-25 00:30:21,24.9,76.06,998.84,5.08,175.9,-90.9653,32.9786,-91.373413,32.680164
...,...,...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.4,89.82,1000.46,7.66,221.62,-90.9653,32.9786,-90.257489,33.227261
5276,2023-03-25 01:58:13,23.4,89.84,1000.46,7.88,221.5,-90.9653,32.9786,-90.257533,33.227391
5277,2023-03-25 01:58:14,23.4,89.86,1000.46,7.78,220.66,-90.9653,32.9786,-90.257576,33.227521
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-90.257619,33.227651


In [8]:
dec_time = []
for t in range(0, len(smooth_5s_A105), 1):
    time = smooth_5s_A105['Time'][t]
    hh = int(time[11:13])
    mm = int(time[14:16])
    ss = int(time[17:19])
    dec = hh + ((mm*60) + ss)/3600
    dec_time.append(dec)
smooth_5s_A105['Decimal Time'] = dec_time

In [9]:
smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat,Meso Lon,Meso Lat,Decimal Time
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-91.374930,32.679903,0.504722
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-91.374551,32.679969,0.505000
2,2023-03-25 00:30:19,24.9,76.02,998.8,5.28,167.06,-90.9653,32.9786,-91.374171,32.680034,0.505278
3,2023-03-25 00:30:20,24.9,76.04,998.82,5.14,169.62,-90.9653,32.9786,-91.373792,32.680099,0.505556
4,2023-03-25 00:30:21,24.9,76.06,998.84,5.08,175.9,-90.9653,32.9786,-91.373413,32.680164,0.505833
...,...,...,...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.4,89.82,1000.46,7.66,221.62,-90.9653,32.9786,-90.257489,33.227261,1.970000
5276,2023-03-25 01:58:13,23.4,89.84,1000.46,7.88,221.5,-90.9653,32.9786,-90.257533,33.227391,1.970278
5277,2023-03-25 01:58:14,23.4,89.86,1000.46,7.78,220.66,-90.9653,32.9786,-90.257576,33.227521,1.970556
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-90.257619,33.227651,1.970833


In [10]:
## Haversine ##

In [11]:
smooth_5s_A105['105A Lon Rad'] = np.radians(smooth_5s_A105['105A Lon'])
smooth_5s_A105['105A Lat Rad'] = np.radians(np.array(smooth_5s_A105['105A Lat']))
smooth_5s_A105['Meso Lon Rad'] = np.radians(smooth_5s_A105['Meso Lon'])
smooth_5s_A105['Meso Lat Rad'] = np.radians(smooth_5s_A105['Meso Lat'])

smooth_5s_A105['a'] = ((np.sin((smooth_5s_A105['105A Lat Rad'] - smooth_5s_A105['Meso Lat Rad']) / 2))**2 
             + np.cos(smooth_5s_A105['Meso Lat Rad'])*np.cos(smooth_5s_A105['105A Lat Rad'])*(np.sin((smooth_5s_A105['105A Lon Rad'] - smooth_5s_A105['Meso Lon Rad'])/2))**2)

smooth_5s_A105['c'] = 2*np.arctan2(np.sqrt(smooth_5s_A105['a']), np.sqrt((1 - smooth_5s_A105['a'])))

smooth_5s_A105['Distance'] = smooth_5s_A105['c']*6371 # unit for Radius of Earth is km

y = np.sin(smooth_5s_A105['105A Lon Rad'] - (smooth_5s_A105['Meso Lon Rad']))*np.cos(smooth_5s_A105['105A Lat Rad'])
x = ((np.cos(smooth_5s_A105['Meso Lat Rad'])*np.sin(smooth_5s_A105['105A Lat Rad'])) 
     - (np.sin(smooth_5s_A105['Meso Lat Rad'])*np.cos(smooth_5s_A105['105A Lat Rad'])*np.cos(smooth_5s_A105['105A Lon Rad'] - smooth_5s_A105['Meso Lon Rad'])))
smooth_5s_A105['Bearing'] = np.arctan2(y, x)

smooth_5s_A105['Bearing Degrees'] = (np.rad2deg(smooth_5s_A105['Bearing']))

smooth_5s_A105['x'] = smooth_5s_A105['Distance']*np.sin(smooth_5s_A105['Bearing'])
smooth_5s_A105['y'] = smooth_5s_A105['Distance']*np.cos(smooth_5s_A105['Bearing'])

smooth_5s_A105

,Time,Temp,RH,Pressure,Wind_Speed,Wind_Direction,105A Lon,105A Lat,Meso Lon,Meso Lat,...,105A Lat Rad,Meso Lon Rad,Meso Lat Rad,a,c,Distance,Bearing,Bearing Degrees,x,y
0,2023-03-25 00:30:17,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-91.374930,32.679903,...,0.575585,-1.594793,0.570372,0.000016,0.007954,50.675859,0.854135,48.938316,38.209741,33.287510
1,2023-03-25 00:30:18,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-91.374551,32.679969,...,0.575585,-1.594787,0.570373,0.000016,0.007949,50.644333,0.853786,48.918338,38.174369,33.280114
2,2023-03-25 00:30:19,24.9,76.02,998.8,5.28,167.06,-90.9653,32.9786,-91.374171,32.680034,...,0.575585,-1.594780,0.570374,0.000016,0.007944,50.612814,0.853437,48.898336,38.138998,33.272719
3,2023-03-25 00:30:20,24.9,76.04,998.82,5.14,169.62,-90.9653,32.9786,-91.373792,32.680099,...,0.575585,-1.594774,0.570375,0.000016,0.007939,50.581302,0.853087,48.878309,38.103627,33.265323
4,2023-03-25 00:30:21,24.9,76.06,998.84,5.08,175.9,-90.9653,32.9786,-91.373413,32.680164,...,0.575585,-1.594767,0.570376,0.000016,0.007934,50.549795,0.852737,48.858256,38.068254,33.257928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,2023-03-25 01:58:12,23.4,89.82,1000.46,7.66,221.62,-90.9653,32.9786,-90.257489,33.227261,...,0.575585,-1.575290,0.579925,0.000031,0.011222,71.493368,-1.964518,-112.558594,-66.023246,-27.426861
5276,2023-03-25 01:58:13,23.4,89.84,1000.46,7.88,221.5,-90.9653,32.9786,-90.257533,33.227391,...,0.575585,-1.575291,0.579927,0.000031,0.011222,71.495193,-1.964727,-112.570556,-66.019203,-27.441344
5277,2023-03-25 01:58:14,23.4,89.86,1000.46,7.78,220.66,-90.9653,32.9786,-90.257576,33.227521,...,0.575585,-1.575292,0.579930,0.000031,0.011222,71.497020,-1.964936,-112.582516,-66.015161,-27.455828
5278,2023-03-25 01:58:15,NaN,NaN,NaN,NaN,NaN,-90.9653,32.9786,-90.257619,33.227651,...,0.575585,-1.575293,0.579932,0.000031,0.011223,71.498851,-1.965144,-112.594477,-66.011118,-27.470311


In [12]:
## Variable Calcs ##

In [13]:
theta = []
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    theta_calc = metpy.calc.potential_temperature(p, t)
    theta.append(theta_calc)

In [14]:
td = []
for i in range(0, len(smooth_5s_A105), 1):
    t = smooth_5s_A105['Temp'][i]*units.degC
    rh = (smooth_5s_A105['RH'][i])*units.percent
    td_calc = metpy.calc.dewpoint_from_relative_humidity(t, rh)
    td.append(td_calc)

In [15]:
smooth_5s_A105['Dew Point'] = td

In [16]:
theta_e = []
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    td = smooth_5s_A105['Dew Point'][i]
    theta_e_calc = metpy.calc.equivalent_potential_temperature(p, t, td)
    theta_e.append(theta_e_calc)

In [17]:
#e = []
#for i in range(0, len(data), 1):
#    td = data['Dew Point'][i]
#    e_calc = 611.2*math.exp((17.67*td)/(td + 243.5))
#    e_calc = (e_calc/100)*units.hPa
#    e.append(e_calc)

In [18]:
#mixing_ratio = [] # in kg/kg, unitless
#molecular_weight_ratio = 0.6219569100577033
#for i in range(0, len(data), 1):
#    partial_pressure = e[i]
#    p = data['Pressure'][i]*units.hPa
#    mix_rat_calc = metpy.calc.mixing_ratio(partial_pressure, p, molecular_weight_ratio)
#    mixing_ratio.append(mix_rat_calc)

In [19]:
mixing_ratio = []
for i in range(0, len(smooth_5s_A105), 1):
    t = smooth_5s_A105['Temp'][i]*units.degC
    rh = (smooth_5s_A105['RH'][i])*units.percent
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    mix_rat_calc = metpy.calc.mixing_ratio_from_relative_humidity(p, t, rh)
    mixing_ratio.append(mix_rat_calc)

In [20]:
theta_v = []
molecular_weight_ratio = 0.6219569100577033
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    mix = mixing_ratio[i]
    theta_v_calc = metpy.calc.virtual_potential_temperature(p, t, mix, molecular_weight_ratio)
    theta_v.append(theta_v_calc)

In [21]:
wind_comp = []
for i in range (0, len(smooth_5s_A105), 1):
    spd = smooth_5s_A105['Wind_Speed'][i]*units('m/s')
    dire = smooth_5s_A105['Wind_Direction'][i]*units.deg
    comp = metpy.calc.wind_components(spd, dire)
    wind_comp.append(comp)

In [22]:
u = []
v = []
for i in range(0, len(wind_comp), 1):
    ui, vi = wind_comp[i]
    u.append(ui)
    v.append(vi)

In [23]:
storm_speed = np.full(5280, 23)
storm_motion = np.full(5280, 241)

In [24]:
wind_comp_storm = []
for i in range (0, len(smooth_5s_A105), 1):
    spd = storm_speed[i]*units('m/s')
    dire = storm_motion[i]*units.deg
    comp = metpy.calc.wind_components(spd, dire)
    wind_comp_storm.append(comp)

In [25]:
u_s = []
v_s = []
for i in range(0, len(wind_comp_storm), 1):
    uj, vj = wind_comp_storm[i]
    u_s.append(uj)
    v_s.append(vj)

In [26]:
altimeter = [] # https://www.weather.gov/media/epz/wxcalc/altimeterSetting.pdf
a = 0.190284
b = 0.0065*1013.25**a
for i in range(0, len(smooth_5s_A105), 1):
    alt_set = ((smooth_5s_A105['Pressure'][i] - 0.3)
               *(1 + ((b/288)*((32)/((smooth_5s_A105['Pressure'][i] - 0.3)**a))))**(1/a))
    altimeter.append(alt_set)

In [27]:
mslp = []
for i in range(0, len(smooth_5s_A105)):
    mslp_calc = metpy.calc.altimeter_to_sea_level_pressure(altimeter[i]*units.hPa, 
                                                           32*units.m, 
                                                           smooth_5s_A105['Temp'][i]*units.degC)
    mslp.append(mslp_calc)

In [28]:
delta_d = [np.nan*units.km]
x_bar = [np.nan]
y_bar = [np.nan]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    x0 = smooth_5s_A105['x'][i - 1]
    y0 = smooth_5s_A105['y'][i - 1]
    x2 = smooth_5s_A105['x'][i + 1]
    y2 = smooth_5s_A105['y'][i + 1]
    dist = (((x2 - x0)**2 + (y2 - y0)**2)**(1/2))*units.km
    x = (x2 + x0)/2
    y = (y2 + y0)/2
    delta_d.append(dist)
    x_bar.append(x)
    y_bar.append(y)
delta_d.append(np.nan*units.km)
x_bar.append(np.nan)
y_bar.append(np.nan)

In [29]:
smooth_5s_A105['Theta'] = theta
smooth_5s_A105['Theta_e'] = theta_e
smooth_5s_A105['Theta_v'] = theta_v
#smooth_5s_A105['Vapor Pressure'] = e
smooth_5s_A105['Mixing Ratio'] = mixing_ratio
smooth_5s_A105['u_wind'] = u
smooth_5s_A105['v_wind'] = v
smooth_5s_A105['Storm Speed'] = storm_speed
smooth_5s_A105['Storm Motion'] = storm_motion
smooth_5s_A105['storm_u'] = u_s
smooth_5s_A105['storm_v'] = v_s
smooth_5s_A105['storm_rel_u'] = smooth_5s_A105['u_wind'] - smooth_5s_A105['storm_u']
smooth_5s_A105['storm_rel_v'] = smooth_5s_A105['v_wind'] - smooth_5s_A105['storm_v']
smooth_5s_A105['Altimeter'] = altimeter
smooth_5s_A105['MSLP'] = mslp
smooth_5s_A105['Delta Probe Distance'] = delta_d
smooth_5s_A105['Gradient Mean x'] = x_bar
smooth_5s_A105['Gradient Mean y'] = y_bar

In [30]:
density = []
mol_weight = 0.62195691
for i in range(0, len(smooth_5s_A105), 1):
    p = smooth_5s_A105['Pressure'][i]*units.hPa
    t = smooth_5s_A105['Temp'][i]*units.degC
    r = smooth_5s_A105['Mixing Ratio'][i]
    den = metpy.calc.density(p, t, r, mol_weight)
    density.append(den)

In [31]:
## Gradient Calculations ##

In [32]:
grad_theta = [np.nan*units('K/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    theta2 = smooth_5s_A105['Theta'][i + 1]
    theta1 = smooth_5s_A105['Theta'][i - 1]
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtheta = (theta2 - theta1) / d
    grad_theta.append(dtheta)
grad_theta.append(np.nan*units('K/km'))

In [33]:
grad_theta_e = [np.nan*units('K/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    theta_e2 = smooth_5s_A105['Theta_e'][i + 1]
    theta_e1 = smooth_5s_A105['Theta_e'][i - 1]
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtheta_e = (theta_e2 - theta_e1) / d
    grad_theta_e.append(dtheta_e)
grad_theta_e.append(np.nan*units('K/km'))

In [34]:
grad_theta_v = [np.nan*units('K/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    theta_v2 = smooth_5s_A105['Theta_v'][i + 1]
    theta_v1 = smooth_5s_A105['Theta_v'][i - 1]
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtheta_v = (theta_v2 - theta_v1) / d
    grad_theta_v.append(dtheta_v)
grad_theta_v.append(np.nan*units('K/km'))

In [35]:
grad_temp = [np.nan*units('degC/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    temp2 = smooth_5s_A105['Temp'][i + 1]*units.degC
    temp1 = smooth_5s_A105['Temp'][i - 1]*units.degC
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dtemp = ((temp2 - temp1) / d)
    grad_temp.append(dtemp)
grad_temp.append(np.nan*units('degC/km'))

In [36]:
grad_press = [np.nan*units('hPa/km')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    press2 = smooth_5s_A105['Pressure'][i + 1]*units.hPa
    press1 = smooth_5s_A105['Pressure'][i - 1]*units.hPa
    d = smooth_5s_A105['Delta Probe Distance'][i]
    dpress = ((press2 - press1) / d)
    grad_press.append(dpress)
grad_press.append(np.nan*units('hPa/km'))

In [37]:
baro_vort = [np.nan*units('1 / s')]
for i in range(1, len(smooth_5s_A105) - 1, 1):
    gen = (((density[i + 1] - density[i - 1])/delta_d[i].to(units.m))
           *grad_press[i].to(units('Pa/m')))*(1/density[i]**2)
    gen = gen*2*units.s
    baro_vort.append(gen.to(units('1 / s')))
baro_vort.append(np.nan*units('1 / s'))

In [38]:
smooth_5s_A105['Density'] = density
smooth_5s_A105['Delta Theta'] = grad_theta
smooth_5s_A105['Delta Theta_e'] = grad_theta_e
smooth_5s_A105['Delta Theta_v'] = grad_theta_v
smooth_5s_A105['Delta Temp'] = grad_temp
smooth_5s_A105['Delta Pressure'] = grad_press
smooth_5s_A105['Baroclinic Generated Vorticity'] = baro_vort

In [39]:
probe_bearing = []
x_comp = []
y_comp = []

probe_bearing.append(np.nan)


for i in range(1, len(smooth_5s_A105) - 1, 1):
    x0 = smooth_5s_A105['x'][i - 1]
    x1 = smooth_5s_A105['x'][i + 1]
    y0 = smooth_5s_A105['y'][i - 1]
    y1 = smooth_5s_A105['y'][i + 1]
    dx = x1 - x0
    dy = y1 - y0
    if (dx < 0) & (dy < 0):
        angle = np.arctan((dx/dy)) + np.pi
        probe_bearing.append(angle)
    elif (dx < 0) & (dy > 0):
        angle = np.arctan((dy/dx)) + ((3*np.pi)/2)
        probe_bearing.append(angle)
    elif (dx > 0) & (dy < 0):
        angle = np.pi - np.arctan((dx/dy))
        probe_bearing.append(angle)
    elif (dx > 0) & (dy > 0):
        angle = (np.pi/2) - np.arctan((dy/dx))
        probe_bearing.append(angle)
    elif (dx == 0) & (dy > 0):
        angle = 0
        probe_bearing.append(angle)
    elif (dx == 0) & (dy < 0):
        angle = np.pi
        probe_bearing.append(angle)
    elif (dx < 0) & (dy == 0):
        angle = (3*np.pi)/2
        probe_bearing.append(angle)
    else : # dx > 0 & dy == 0
        angle = np.pi/2
        probe_bearing.append(angle)
probe_bearing.append(np.nan)

for j in range(0, len(smooth_5s_A105) - 1, 1):
    mag = smooth_5s_A105['Baroclinic Generated Vorticity'][j]
    if (mag > 0):
        mag_bear = probe_bearing[j] - np.pi/2
        u = np.abs(mag)*np.sin(mag_bear)
        v = np.abs(mag)*np.cos(mag_bear)
        x_comp.append(u)
        y_comp.append(v)
    elif (mag < 0):
        mag_bear = probe_bearing[j] + np.pi/2
        u = np.abs(mag)*np.sin(mag_bear)
        v = np.abs(mag)*np.cos(mag_bear)
        x_comp.append(u)
        y_comp.append(v)
    else : # mag == 0
        x_comp.append(np.nan)
        y_comp.append(np.nan)

x_comp.append(np.nan)
y_comp.append(np.nan)



In [40]:
smooth_5s_A105['Probe Bearing'] = probe_bearing
smooth_5s_A105['BGV x_comp'] = x_comp
smooth_5s_A105['BGV y_comp'] = y_comp

In [41]:
wind_bearing = []
bgv_bearing = []

for i in range(0, len(smooth_5s_A105), 1):
    u = smooth_5s_A105['storm_rel_u'][i]
    v = smooth_5s_A105['storm_rel_v'][i]
    if (u < 0) & (v < 0):
        angle = np.arctan((u/v)) + np.pi
        wind_bearing.append(angle)
    elif (u < 0) & (v > 0):
        angle = np.arctan((v/u)) + ((3*np.pi)/2)
        wind_bearing.append(angle)
    elif (u > 0) & (v < 0):
        angle = np.pi - np.arctan((u/v))
        wind_bearing.append(angle)
    elif (u > 0) & (v > 0):
        angle = (np.pi/2) - np.arctan((v/u))
        wind_bearing.append(angle)
    elif (u == 0) & (v > 0):
        angle = 0
        wind_bearing.append(angle)
    elif (u == 0) & (v < 0):
        angle = np.pi
        wind_bearing.append(angle)
    elif (u < 0) & (v == 0):
        angle = (3*np.pi)/2
        wind_bearing.append(angle)
    else : # dx > 0 & dy == 0
        angle = np.pi/2
        wind_bearing.append(angle)

for i in range(0, len(smooth_5s_A105), 1):
    u = smooth_5s_A105['BGV x_comp'][i]
    v = smooth_5s_A105['BGV y_comp'][i]
    if (u < 0) & (v < 0):
        angle = np.arctan((u/v)) + np.pi
        bgv_bearing.append(angle)
    elif (u < 0) & (v > 0):
        angle = np.arctan((v/u)) + ((3*np.pi)/2)
        bgv_bearing.append(angle)
    elif (u > 0) & (v < 0):
        angle = np.pi - np.arctan((u/v))
        bgv_bearing.append(angle)
    elif (u > 0) & (v > 0):
        angle = (np.pi/2) - np.arctan((v/u))
        bgv_bearing.append(angle)
    elif (u == 0) & (v > 0):
        angle = 0
        bgv_bearing.append(angle)
    elif (u == 0) & (v < 0):
        angle = np.pi
        bgv_bearing.append(angle)
    elif (u < 0) & (v == 0):
        angle = (3*np.pi)/2
        bgv_bearing.append(angle)
    else : # dx > 0 & dy == 0
        angle = np.pi/2
        bgv_bearing.append(angle)

        
streamwise_vort_comp = []
crosswise_vort_comp = []

for i in range(0, len(smooth_5s_A105), 1):
    vort_mag = smooth_5s_A105['Baroclinic Generated Vorticity'][i]
    wind_angle = wind_bearing[i]
    bvg_angle = bgv_bearing[i]
    alpha = wind_angle - bvg_angle
    sw = vort_mag*np.cos(alpha)
    cw = vort_mag*np.sin(alpha)
    streamwise_vort_comp.append(sw)
    crosswise_vort_comp.append(cw)

In [42]:
smooth_5s_A105['Wind Bearing from N'] = wind_bearing
smooth_5s_A105['BGV Bearing from N'] = bgv_bearing
smooth_5s_A105['Streamwise Vort Mag'] = streamwise_vort_comp
smooth_5s_A105['Crosswise Vort Mag'] = crosswise_vort_comp

In [43]:
smooth_5s_A105.to_csv('/Users/josia/Desktop/Python/StickNet/106A_Data_Final.csv', index = False)